Potrzebne importy

In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
import keras

from keras import models
from keras import layers
from keras.datasets import mnist
from keras.utils import to_categorical
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV, StratifiedKFold
from keras.wrappers.scikit_learn import KerasClassifier


Powtórzyć trening klasycznych klasyfikatorów i sieci dla danych bez wyrównania (dst)

In [ ]:
SZ=28
affine_flags = cv2.WARP_INVERSE_MAP|cv2.INTER_LINEAR

Deskew

In [ ]:
def deskew(img):
    m = cv2.moments(img)
    if abs(m['mu02']) < 1e-2:
        return img.copy()
    skew = m['mu11']/m['mu02']
    M = np.float32([[1, skew, -0.5*SZ*skew], [0, 1, 0]])
    img = cv2.warpAffine(img,M,(SZ, SZ),flags=affine_flags)
    return img

In [ ]:
def showOpencvImage(image, isGray=False):
    fig = plt.figure(figsize=(6, 6))
    plt.imshow(image, cmap = 'gray')
    plt.show()


def openCVHOG(im):
    winSize = (20,20)
    blockSize = (10,10)
    blockStride = (5,5)
    cellSize = (10,10)
    nbins = 9
    derivAperture = 1
    winSigma = -1.
    histogramNormType = 0
    L2HysThreshold = 0.2
    gammaCorrection = 1
    nlevels = 64
    signedGradients = True

    hog = cv2.HOGDescriptor(winSize,blockSize,blockStride,cellSize,nbins,derivAperture,winSigma,histogramNormType,L2HysThreshold,gammaCorrection,nlevels, signedGradients)
    descriptor = np.ravel(hog.compute(im))
    
    return descriptor

In [ ]:
def create_model():
      model = models.Sequential()
      model.add(layers.Dense(512, activation='relu')) 
      model.add(layers.Dense(10, activation='softmax'))      
      model.compile(optimizer='rmsprop',
                      loss='categorical_crossentropy',
                      metrics=['accuracy'])
      return model

Podział zbioru

In [ ]:
(train_images, train_labels), (test_images, test_labels) = mnist.load_data()

Przygotowanie danych wejsciowych

In [ ]:
# dane surowe
train_raw_data = train_images.reshape(len(train_images), 28 * 28)
test_raw_data = test_images.reshape(len(test_images), 28 * 28)

#hogdata bez wyrownania

hogdata_train = np.float32([openCVHOG(im) for im in train_images]).reshape(-1,81)
hogdata_test = np.float32([openCVHOG(im) for im in test_images]).reshape(-1,81)

#hogdata po wyrownaniu
hogdata_train_deskewed = np.float32([openCVHOG(deskew(im)) for im in train_images]).reshape(-1,81)
hogdata_test_deskewed = np.float32([openCVHOG(deskew(im)) for im in test_images]).reshape(-1,81)

In [ ]:
#wyciecie etykiet
train_labels_short = train_labels[0:600]
test_labels_short=test_labels[600:1000]


#dla sieci neuronowych
encoded_train_labels_short = to_categorical(train_labels_short)
encoded_test_labels_short = to_categorical(test_labels_short)

In [ ]:
#wyciecie danych

#dane bez wyrówania
hogdata_train_short=hogdata_train[:600,]
hogdata_test_short=hogdata_test[600:1000,:]

#dane z wyrównaniem
deskewed_hogdata_train_short=hogdata_train_deskewed[:600,]
deskewed_hogdata_test_short=hogdata_test_deskewed[600:1000,:]

#dane surowe
train_raw_data_short=train_raw_data[:600,]
test_raw_data_short=test_raw_data[600:1000,:]


In [ ]:
#dane surowe zmieszane
permute_train_raw_data = np.random.permutation(train_raw_data)
permute_test_raw_data =np.random.permutation(test_raw_data)

permute_train_raw_data_short=permute_train_raw_data[:600,]
permute_test_raw_data_short=permute_test_raw_data[600:1000,:]

# SVM

Model SVM dla:

- danych bez wyrówania
- danych z wyrównaniem
- danych surowych

Dla danych bez wyrównania:

In [ ]:
parameters = {'kernel':('linear', 'rbf'), 'C': np.linspace(start = 0.001, stop = 2, num = 200)}
svc = SVC()
model_svc = GridSearchCV(svc, parameters, scoring='accuracy', cv=StratifiedKFold())
model_svc.fit(hogdata_train_short,train_labels_short)
params = model_svc.best_params_
my_svc = model_svc.best_estimator_
print("Wybrane parametry: ", params)
print("Wybrany estymator: ", my_svc)

pred_labels_svc = model_svc.predict(hogdata_test_short)
mask = pred_labels_svc==test_labels_short


Wybrane parametry:  {'C': 0.864889447236181, 'kernel': 'linear'}
Wybrany estymator:  SVC(C=0.864889447236181, break_ties=False, cache_size=200, class_weight=None,
    coef0=0.0, decision_function_shape='ovr', degree=3, gamma='scale',
    kernel='linear', max_iter=-1, probability=False, random_state=None,
    shrinking=True, tol=0.001, verbose=False)


Raport:

In [ ]:
print("Accuracy dla SVC bez wyrowania: ", accuracy_score(test_labels_short, pred_labels_svc))
print("Confusion matrix dla SVC (dane bez wyrownania): \n")
print(confusion_matrix(test_labels_short, pred_labels_svc))
print("Classification report dlaSVC (dane bez wyrowanania): ")
print(classification_report(test_labels_short, pred_labels_svc))

Accuracy dla SVC bez wyrowania:  0.89
Confusion matrix dla SVC (dane bez wyrownania): 

[[31  0  0  0  0  1  0  0  0  0]
 [ 0 53  0  0  0  0  0  0  0  0]
 [ 0  1 44  0  0  0  0  5  1  1]
 [ 0  0  2 41  0  2  0  0  0  0]
 [ 0  0  0  0 41  0  0  0  0  2]
 [ 0  1  0  0  0 27  0  0  1  2]
 [ 0  0  0  0  0  0 35  0  0  0]
 [ 0  0  5  4  0  1  0 31  0  1]
 [ 1  0  1  1  0  2  1  2 24  5]
 [ 0  0  1  0  0  0  0  0  0 29]]
Classification report dlaSVC (dane bez wyrowanania): 
              precision    recall  f1-score   support

           0       0.97      0.97      0.97        32
           1       0.96      1.00      0.98        53
           2       0.83      0.85      0.84        52
           3       0.89      0.91      0.90        45
           4       1.00      0.95      0.98        43
           5       0.82      0.87      0.84        31
           6       0.97      1.00      0.99        35
           7       0.82      0.74      0.78        42
           8       0.92      0.65      0

SVC dane z wyrownaniem:

In [ ]:
parameters = {'kernel':('linear', 'rbf'), 'C': np.linspace(start = 0.001, stop = 2, num = 200)}
svc = SVC()
model = GridSearchCV(svc, parameters, scoring='accuracy', cv=StratifiedKFold())
model.fit(deskewed_hogdata_train_short,train_labels_short)
params = model.best_params_
my_svc = model.best_estimator_
print("Wybrane parametry: ", params)
print("Wybrany estymator: ", my_svc)

pred_labels = model.predict(deskewed_hogdata_test_short)
mask = pred_labels==test_labels_short

Wybrane parametry:  {'C': 0.714211055276382, 'kernel': 'linear'}
Wybrany estymator:  SVC(C=0.714211055276382, break_ties=False, cache_size=200, class_weight=None,
    coef0=0.0, decision_function_shape='ovr', degree=3, gamma='scale',
    kernel='linear', max_iter=-1, probability=False, random_state=None,
    shrinking=True, tol=0.001, verbose=False)


In [ ]:
print("Accuracy dla SVC z wyrownaniem: ", accuracy_score(test_labels_short, pred_labels))
print("Confusion matrix SVC (dane z wyrowaniem): \n")
print(confusion_matrix(test_labels_short, pred_labels))
print("Classification report SVC (dane z wyrowaniem): ")
print(classification_report(test_labels_short, pred_labels))

Accuracy dla SVC z wyrownaniem:  0.9125
Confusion matrix SVC (dane z wyrowaniem): 

[[32  0  0  0  0  0  0  0  0  0]
 [ 0 52  0  0  0  0  0  1  0  0]
 [ 0  1 48  0  0  0  0  2  1  0]
 [ 0  0  3 37  0  2  0  0  2  1]
 [ 0  0  0  0 42  0  0  1  0  0]
 [ 0  0  0  0  0 29  0  1  1  0]
 [ 0  0  0  0  0  0 35  0  0  0]
 [ 1  0  4  4  0  1  0 32  0  0]
 [ 0  0  1  0  0  2  0  0 31  3]
 [ 0  0  1  0  0  0  0  2  0 27]]
Classification report SVC (dane z wyrowaniem): 
              precision    recall  f1-score   support

           0       0.97      1.00      0.98        32
           1       0.98      0.98      0.98        53
           2       0.84      0.92      0.88        52
           3       0.90      0.82      0.86        45
           4       1.00      0.98      0.99        43
           5       0.85      0.94      0.89        31
           6       1.00      1.00      1.00        35
           7       0.82      0.76      0.79        42
           8       0.89      0.84      0.86       

SVC dla surowych danych:

In [ ]:
parameters = {'kernel':('linear', 'rbf'), 'C': np.linspace(start = 0.001, stop = 2, num = 200)}
svc = SVC()
model = GridSearchCV(svc, parameters, scoring='accuracy', cv=StratifiedKFold())
model.fit(train_raw_data_short,train_labels_short)
params = model.best_params_
my_svc = model.best_estimator_
print("Chosing params: ", params)
print("Chosing estimator: ", my_svc)

pred_labels = model.predict(test_raw_data_short)
mask = pred_labels==test_labels_short

Chosing params:  {'C': 1.2666984924623115, 'kernel': 'rbf'}
Chosing estimator:  SVC(C=1.2666984924623115, break_ties=False, cache_size=200, class_weight=None,
    coef0=0.0, decision_function_shape='ovr', degree=3, gamma='scale',
    kernel='rbf', max_iter=-1, probability=False, random_state=None,
    shrinking=True, tol=0.001, verbose=False)


In [ ]:
print("Accuracy dla SVC (surowe dane): ", accuracy_score(test_labels_short, pred_labels))
print("Confusion matrix dla SVC (surowe dane): \n")
print(confusion_matrix(test_labels_short, pred_labels))
print("Classification report dla SVC (surowe dane): ")
print(classification_report(test_labels_short, pred_labels))

Accuracy dla SVC (surowe dane):  0.8675
Confusion matrix dla SVC (surowe dane): 

[[30  0  0  0  0  1  1  0  0  0]
 [ 0 52  1  0  0  0  0  0  0  0]
 [ 1  1 43  1  0  0  0  3  3  0]
 [ 0  0  0 36  0  7  0  0  1  1]
 [ 0  0  0  0 35  0  0  0  0  8]
 [ 1  0  0  0  1 27  0  1  0  1]
 [ 1  0  1  0  0  1 32  0  0  0]
 [ 0  0  2  1  1  0  0 36  0  2]
 [ 0  0  1  2  0  2  0  1 30  1]
 [ 0  0  1  0  2  0  0  1  0 26]]
Classification report dla SVC (surowe dane): 
              precision    recall  f1-score   support

           0       0.91      0.94      0.92        32
           1       0.98      0.98      0.98        53
           2       0.88      0.83      0.85        52
           3       0.90      0.80      0.85        45
           4       0.90      0.81      0.85        43
           5       0.71      0.87      0.78        31
           6       0.97      0.91      0.94        35
           7       0.86      0.86      0.86        42
           8       0.88      0.81      0.85        37


SVC dla surowych danych zmieszanych:

In [ ]:
parameters = {'kernel':('linear', 'rbf'), 'C': np.linspace(start = 0.001, stop = 2, num = 200)}
svc = SVC()
model = GridSearchCV(svc, parameters, scoring='accuracy', cv=StratifiedKFold())
model.fit(permute_train_raw_data_short,train_labels_short)
params = model.best_params_
my_svc = model.best_estimator_
print("Chosing params: ", params)
print("Chosing estimator: ", my_svc)

pred_labels = model.predict(permute_test_raw_data_short)
mask = pred_labels==test_labels_short

Chosing params:  {'C': 0.001, 'kernel': 'rbf'}
Chosing estimator:  SVC(C=0.001, break_ties=False, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='scale', kernel='rbf',
    max_iter=-1, probability=False, random_state=None, shrinking=True,
    tol=0.001, verbose=False)


In [ ]:
print("Accuracy dla SVC (surowe dane zmieszane): ", accuracy_score(test_labels_short, pred_labels))
print("Confusion matrix dla SVC (surowe dane zmieszane): \n")
print(confusion_matrix(test_labels_short, pred_labels))
print("Classification report dla SVC (surowe dane zmieszane): ")
print(classification_report(test_labels_short, pred_labels))

Accuracy dla SVC (surowe dane zmieszane):  0.1325
Confusion matrix dla SVC (surowe dane zmieszane): 

[[ 0 32  0  0  0  0  0  0  0  0]
 [ 0 53  0  0  0  0  0  0  0  0]
 [ 0 52  0  0  0  0  0  0  0  0]
 [ 0 45  0  0  0  0  0  0  0  0]
 [ 0 43  0  0  0  0  0  0  0  0]
 [ 0 31  0  0  0  0  0  0  0  0]
 [ 0 35  0  0  0  0  0  0  0  0]
 [ 0 42  0  0  0  0  0  0  0  0]
 [ 0 37  0  0  0  0  0  0  0  0]
 [ 0 30  0  0  0  0  0  0  0  0]]
Classification report dla SVC (surowe dane zmieszane): 
              precision    recall  f1-score   support

           0       0.00      0.00      0.00        32
           1       0.13      1.00      0.23        53
           2       0.00      0.00      0.00        52
           3       0.00      0.00      0.00        45
           4       0.00      0.00      0.00        43
           5       0.00      0.00      0.00        31
           6       0.00      0.00      0.00        35
           7       0.00      0.00      0.00        42
           8       0.00 

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


# RandomFores

RandomForest bez wyrównania

In [ ]:
param_grid = {'max_depth': [5,10, 15, 20, 50, None], 'n_estimators':[20, 50, 60, 100, 120], 'max_features': [60]}
model_rf = GridSearchCV(RandomForestClassifier(), param_grid=param_grid, scoring='accuracy')

model_rf_non_deskewed = model_rf.fit(hogdata_train_short, train_labels_short)
pred_labels_rf_non_deskewed = model_rf_non_deskewed.predict(hogdata_test_short)


RandomForest z wyrównaniem:

In [ ]:
param_grid = {'max_depth': [5,10, 15, 20, 50, None], 'n_estimators':[20, 50, 60, 100, 120], 'max_features': [60]}
model_rf = GridSearchCV(RandomForestClassifier(), param_grid=param_grid, scoring='accuracy')

model_rf_deskewed = model_rf.fit(deskewed_hogdata_train_short, train_labels_short)
pred_labels_rf_deskewed = model_rf_deskewed.predict(deskewed_hogdata_test_short)


In [ ]:
print(model_rf_deskewed.best_params_)

{'max_depth': 15, 'max_features': 60, 'n_estimators': 100}


Raport:

In [ ]:
print("Accuracy dla wyrownanych obrazow: " + str(accuracy_score(test_labels_short, pred_labels_rf_deskewed)))
print("Accuracy dla niewyrownanych: " + str(accuracy_score(test_labels_short, pred_labels_rf_non_deskewed)))

Accuracy dla wyrownanych obrazow: 0.87
Accuracy dla niewyrownanych: 0.8275


In [ ]:
cmatrix_rf_deskewed = confusion_matrix(test_labels_short, pred_labels_rf_deskewed)
cmatrix_rf_non_deskewed = confusion_matrix(test_labels_short, pred_labels_rf_non_deskewed)

print("Confusion matrix dla wyrownania:")
print(cmatrix_rf_deskewed)
print("\n Classification report dla wyrownania: ") 
print(classification_report(test_labels_short, pred_labels_rf_deskewed))
print("\n Confusion matrix dla niewyrownananych obrazow:")
print(cmatrix_rf_non_deskewed)
print("\n Classification report dla niewyrownananych obrazow: ") 
print(classification_report(test_labels_short, pred_labels_rf_non_deskewed))

Confusion matrix dla wyrownania:
[[31  0  0  1  0  0  0  0  0  0]
 [ 0 52  0  0  0  0  0  1  0  0]
 [ 0  2 43  0  0  1  0  5  0  1]
 [ 0  0  1 35  0  2  0  1  6  0]
 [ 0  0  0  1 40  0  1  1  0  0]
 [ 1  0  0  1  0 27  1  0  1  0]
 [ 0  0  0  0  0  1 34  0  0  0]
 [ 0  0  6  2  1  0  0 31  1  1]
 [ 0  0  0  2  0  2  0  0 28  5]
 [ 0  0  1  0  0  0  0  0  2 27]]

 Classification report dla wyrownania: 
              precision    recall  f1-score   support

           0       0.97      0.97      0.97        32
           1       0.96      0.98      0.97        53
           2       0.84      0.83      0.83        52
           3       0.83      0.78      0.80        45
           4       0.98      0.93      0.95        43
           5       0.82      0.87      0.84        31
           6       0.94      0.97      0.96        35
           7       0.79      0.74      0.77        42
           8       0.74      0.76      0.75        37
           9       0.79      0.90      0.84        30


RandomForest dla wektorów surowych danych 28*28 (zad 3):

In [ ]:
rf_model_raw = model_rf.fit(train_raw_data_short, train_labels_short)
rf_pred_labels_raw = rf_model_raw.predict(test_raw_data_short)
print("Accuracy dla surowych danych: " + str(accuracy_score(test_labels_short, rf_pred_labels_raw)))


Accuracy dla surowych danych: 0.8175


In [ ]:
rf_cmatrix_raw = confusion_matrix(test_labels_short, rf_pred_labels_raw)
print("Confusion matrix dla surowych danych:")
print(rf_cmatrix_raw)
print("\n Classification report dla surowych danych: ") 
print(classification_report(test_labels_short, rf_pred_labels_raw))

Confusion matrix dla surowych danych:
[[30  0  0  0  0  0  2  0  0  0]
 [ 0 50  0  2  0  0  0  0  1  0]
 [ 1  3 40  1  2  1  0  1  2  1]
 [ 0  1  1 33  0  6  1  0  2  1]
 [ 0  0  1  0 34  0  0  0  0  8]
 [ 1  0  0  0  2 26  0  1  0  1]
 [ 1  0  1  0  1  2 30  0  0  0]
 [ 0  0  2  0  1  0  0 35  0  4]
 [ 0  0  1  6  0  2  0  1 24  3]
 [ 0  0  1  0  2  0  0  2  0 25]]

 Classification report dla surowych danych: 
              precision    recall  f1-score   support

           0       0.91      0.94      0.92        32
           1       0.93      0.94      0.93        53
           2       0.85      0.77      0.81        52
           3       0.79      0.73      0.76        45
           4       0.81      0.79      0.80        43
           5       0.70      0.84      0.76        31
           6       0.91      0.86      0.88        35
           7       0.88      0.83      0.85        42
           8       0.83      0.65      0.73        37
           9       0.58      0.83      0.68 

RandomForest dla surowych danych zmieszanych



In [ ]:
rf_model_shuffle = model_rf.fit(permute_train_raw_data_short, train_labels_short)
rf_pred_labels_shuffle = rf_model_raw.predict(permute_test_raw_data_short)
print("Accuracy dla surowych danych: " + str(accuracy_score(test_labels_short, rf_pred_labels_shuffle)))


Accuracy dla surowych danych: 0.1


In [ ]:
rf_cmatrix_shuffle = confusion_matrix(test_labels_short, rf_pred_labels_shuffle)
print("Confusion matrix dla surowych danych:")
print(rf_cmatrix_shuffle)
print("\n Classification report dla surowych danych: ") 
print(classification_report(test_labels_short, rf_pred_labels_shuffle))

Confusion matrix dla surowych danych:
[[ 6  7  3  4  3  3  0  4  0  2]
 [12  9  5  5  5  5  4  1  1  6]
 [ 5 11  7  5  7  4  2  5  4  2]
 [ 8  5  5  3  6  3  5  6  1  3]
 [ 4  3  6  8  4  4  6  3  3  2]
 [ 7  3  1  3  3  0  2  3  4  5]
 [ 6  8  7  4  2  2  1  4  1  0]
 [ 1 11  5  0  6  4  3  7  3  2]
 [ 5  9  5  3  2  2  2  3  2  4]
 [ 4  6  2  2  4  3  2  6  0  1]]

 Classification report dla surowych danych: 
              precision    recall  f1-score   support

           0       0.10      0.19      0.13        32
           1       0.12      0.17      0.14        53
           2       0.15      0.13      0.14        52
           3       0.08      0.07      0.07        45
           4       0.10      0.09      0.09        43
           5       0.00      0.00      0.00        31
           6       0.04      0.03      0.03        35
           7       0.17      0.17      0.17        42
           8       0.11      0.05      0.07        37
           9       0.04      0.03      0.04 

# Sieci Neuronowe

Sieci Neuronowe dla niewyrównanych danych:

In [ ]:
#wektoryzacja

nn_train_non_deskewed = np.array(hogdata_train).reshape((len(train_images), 81))
nn_train_non_deskewed = nn_train_non_deskewed.astype('float32') / 255

nn_test_non_deskewed = np.array(hogdata_test).reshape((len(test_images), 81))
nn_test_non_deskewed = nn_test_non_deskewed.astype('float32') / 255

nn_train_non_deskewed_short = nn_train_non_deskewed[0:600,]
nn_test_non_deskewed_short = nn_test_non_deskewed[600:1000,]



In [ ]:

model=KerasClassifier(create_model)
# param_grid = dict(epochs=[10,20,30])
batch_size = [5,10, 20]
epochs = [1,5,10,15]

param_grid = dict(batch_size=batch_size, epochs=epochs)
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1, cv=3)

grid.fit(nn_train_non_deskewed_short,encoded_train_labels_short)


Epoch 1/15
120/120 [==============================] - 1s 1ms/step - loss: 2.3017 - accuracy: 0.1149
Epoch 2/15
120/120 [==============================] - 0s 1ms/step - loss: 2.2887 - accuracy: 0.1526
Epoch 3/15
120/120 [==============================] - 0s 1ms/step - loss: 2.2846 - accuracy: 0.1340
Epoch 4/15
120/120 [==============================] - 0s 1ms/step - loss: 2.2803 - accuracy: 0.1343
Epoch 5/15
120/120 [==============================] - 0s 1ms/step - loss: 2.2721 - accuracy: 0.1310
Epoch 6/15
120/120 [==============================] - 0s 1ms/step - loss: 2.2498 - accuracy: 0.1296
Epoch 7/15
120/120 [==============================] - 0s 1ms/step - loss: 2.2323 - accuracy: 0.1715
Epoch 8/15
120/120 [==============================] - 0s 1ms/step - loss: 2.1961 - accuracy: 0.1792
Epoch 9/15
120/120 [==============================] - 0s 1ms/step - loss: 2.1799 - accuracy: 0.2317
Epoch 10/15
120/120 [==============================] - 0s 1ms/step - loss: 2.1713 - accuracy: 0.2398

GridSearchCV(cv=3, error_score=nan,
             estimator=<tensorflow.python.keras.wrappers.scikit_learn.KerasClassifier object at 0x7f2f33224f50>,
             iid='deprecated', n_jobs=-1,
             param_grid={'batch_size': [5, 10, 20], 'epochs': [1, 5, 10, 15]},
             pre_dispatch='2*n_jobs', refit=True, return_train_score=False,
             scoring=None, verbose=0)

In [ ]:
print(grid.best_params_)
y_pred = grid.predict(nn_test_non_deskewed_short)

{'batch_size': 5, 'epochs': 15}


/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/sequential.py:450: UserWarning: `model.predict_classes()` is deprecated and will be removed after 2021-01-01. Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
  warnings.warn('`model.predict_classes()` is deprecated and '


In [ ]:
print("Accuracy : ", accuracy_score(test_labels_short, y_pred))
print("Confusion matrix dla Sieci Neuronowych niewyrownanych: \n")
print(confusion_matrix(test_labels_short, y_pred))
print("Classification report dla Sieci Neuronowych niewyrownanych: ")
print(classification_report(test_labels_short, y_pred))

Accuracy :  0.4225
Confusion matrix dla Sieci Neuronowych niewyrownanych: 

[[16  9  0  0  0  0  0  2  0  5]
 [ 0 53  0  0  0  0  0  0  0  0]
 [ 0  8 32  1  0  0  0  9  0  2]
 [ 0  3 17  5  0  0  0 19  0  1]
 [ 0 18  0  0 14  0  0  0  0 11]
 [ 0  4  1  0  0  0  0 12  0 14]
 [10 20  0  0  2  0  2  0  0  1]
 [ 0  2 10  0  0  0  0 27  0  3]
 [ 1  9  4  0  0  0  0 13  0 10]
 [ 1  1  0  0  0  0  0  8  0 20]]
Classification report dla Sieci Neuronowych niewyrownanych: 
              precision    recall  f1-score   support

           0       0.57      0.50      0.53        32
           1       0.42      1.00      0.59        53
           2       0.50      0.62      0.55        52
           3       0.83      0.11      0.20        45
           4       0.88      0.33      0.47        43
           5       0.00      0.00      0.00        31
           6       1.00      0.06      0.11        35
           7       0.30      0.64      0.41        42
           8       0.00      0.00      0.00  

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Sieci Neuronowe dla wyrównanych danych:

In [ ]:
#wektoryzacja

nn_train_deskewed = np.array(hogdata_train_deskewed).reshape((60000, 81))
nn_train_deskewed = nn_train_deskewed.astype('float32') / 255

nn_test_deskewed = np.array(hogdata_test_deskewed).reshape((10000, 81))
nn_test_deskewed = nn_test_deskewed.astype('float32') / 255

nn_train_deskewed_short = nn_train_deskewed[0:600,]
nn_test_deskewed_short = nn_test_deskewed[600:1000,]

In [ ]:

model=KerasClassifier(create_model)
# param_grid = dict(epochs=[10,20,30])
batch_size = [5,10, 20]
epochs = [1,5,10,15]

param_grid = dict(batch_size=batch_size, epochs=epochs)
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1, cv=3)

grid.fit(nn_train_deskewed_short,encoded_train_labels_short)
# grid.fit(nn_train_deskewed,to_categorical(train_labels))

Epoch 1/15
120/120 [==============================] - 1s 1ms/step - loss: 2.3007 - accuracy: 0.1118
Epoch 2/15
120/120 [==============================] - 0s 1ms/step - loss: 2.2884 - accuracy: 0.1465
Epoch 3/15
120/120 [==============================] - 0s 1ms/step - loss: 2.2774 - accuracy: 0.1432
Epoch 4/15
120/120 [==============================] - 0s 1ms/step - loss: 2.2702 - accuracy: 0.1260
Epoch 5/15
120/120 [==============================] - 0s 1ms/step - loss: 2.2419 - accuracy: 0.1546
Epoch 6/15
120/120 [==============================] - 0s 1ms/step - loss: 2.2183 - accuracy: 0.2127
Epoch 7/15
120/120 [==============================] - 0s 1ms/step - loss: 2.1838 - accuracy: 0.2177
Epoch 8/15
120/120 [==============================] - 0s 1ms/step - loss: 2.1277 - accuracy: 0.2735
Epoch 9/15
120/120 [==============================] - 0s 1ms/step - loss: 2.0936 - accuracy: 0.2582
Epoch 10/15
120/120 [==============================] - 0s 1ms/step - loss: 2.0425 - accuracy: 0.3107

GridSearchCV(cv=3, error_score=nan,
             estimator=<tensorflow.python.keras.wrappers.scikit_learn.KerasClassifier object at 0x7f2f332e9d90>,
             iid='deprecated', n_jobs=-1,
             param_grid={'batch_size': [5, 10, 20], 'epochs': [1, 5, 10, 15]},
             pre_dispatch='2*n_jobs', refit=True, return_train_score=False,
             scoring=None, verbose=0)

In [ ]:
print(grid.best_params_)
y_pred = grid.predict(nn_test_deskewed_short)

print("Accuracy : ", accuracy_score(test_labels_short, y_pred))
print("Confusion matrix dla Sieci Neuronowych (wyrowane dane): \n")
print(confusion_matrix(test_labels_short, y_pred))
print("Classification report dla Sieci Neuronowych (wyrownane dane): ")
print(classification_report(test_labels_short, y_pred))


{'batch_size': 5, 'epochs': 15}
Accuracy :  0.485
Confusion matrix dla Sieci Neuronowych (wyrowane dane): 

[[ 7  1  0  0  0  0 12  0  0 12]
 [ 0 53  0  0  0  0  0  0  0  0]
 [ 0  1 48  0  0  0  0  0  0  3]
 [ 0  1 40  0  0  0  0  0  0  4]
 [ 0  2  0  0 24  0 11  0  0  6]
 [ 0  1  1  1  1  0  0  0  0 27]
 [ 0  2  0  0  1  0 32  0  0  0]
 [ 0  0 31  0  3  0  0  3  0  5]
 [ 0  0  7  0  7  0  1  0  0 22]
 [ 0  0  0  0  2  0  1  0  0 27]]
Classification report dla Sieci Neuronowych (wyrownane dane): 
              precision    recall  f1-score   support

           0       1.00      0.22      0.36        32
           1       0.87      1.00      0.93        53
           2       0.38      0.92      0.54        52
           3       0.00      0.00      0.00        45
           4       0.63      0.56      0.59        43
           5       0.00      0.00      0.00        31
           6       0.56      0.91      0.70        35
           7       1.00      0.07      0.13        42
           

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/sequential.py:450: UserWarning: `model.predict_classes()` is deprecated and will be removed after 2021-01-01. Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
  warnings.warn('`model.predict_classes()` is deprecated and '
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Sieci neuronowe dla danych surowych:

In [ ]:
model=KerasClassifier(create_model)
batch_size = [5,10, 20]
epochs = [1,5,10,15]

param_grid = dict(batch_size=batch_size, epochs=epochs)
nn_model_raw = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1, cv=3)
nn_model_raw.fit(train_raw_data_short,encoded_train_labels_short)


Epoch 1/15
120/120 [==============================] - 1s 4ms/step - loss: 79.4667 - accuracy: 0.5177
Epoch 2/15
120/120 [==============================] - 1s 4ms/step - loss: 9.7390 - accuracy: 0.8524
Epoch 3/15
120/120 [==============================] - 1s 4ms/step - loss: 7.3870 - accuracy: 0.8927
Epoch 4/15
120/120 [==============================] - 1s 4ms/step - loss: 2.5474 - accuracy: 0.9479
Epoch 5/15
120/120 [==============================] - 1s 4ms/step - loss: 2.7966 - accuracy: 0.9527
Epoch 6/15
120/120 [==============================] - 1s 4ms/step - loss: 1.8540 - accuracy: 0.9639
Epoch 7/15
120/120 [==============================] - 1s 4ms/step - loss: 1.5772 - accuracy: 0.9675
Epoch 8/15
120/120 [==============================] - 1s 4ms/step - loss: 0.6812 - accuracy: 0.9829
Epoch 9/15
120/120 [==============================] - 1s 4ms/step - loss: 0.1032 - accuracy: 0.9976
Epoch 10/15
120/120 [==============================] - 0s 4ms/step - loss: 0.1918 - accuracy: 0.995

GridSearchCV(cv=3, error_score=nan,
             estimator=<tensorflow.python.keras.wrappers.scikit_learn.KerasClassifier object at 0x7f2f25f96fd0>,
             iid='deprecated', n_jobs=-1,
             param_grid={'batch_size': [5, 10, 20], 'epochs': [1, 5, 10, 15]},
             pre_dispatch='2*n_jobs', refit=True, return_train_score=False,
             scoring=None, verbose=0)

In [ ]:
print(nn_model_raw.best_params_)
nn_pred_raw = nn_model_raw.predict(test_raw_data_short)

print("Accuracy dla Sieci Neuronowych (surowe dane): ", accuracy_score(test_labels_short, nn_pred_raw))
print("Confusion matrix dla Sieci Neuronowych (surowe dane): \n")
print(confusion_matrix(test_labels_short, nn_pred_raw))
print("Classification report dla Sieci Neuronowych (surowe dane): ")
print(classification_report(test_labels_short, nn_pred_raw))

{'batch_size': 5, 'epochs': 15}
Accuracy dla Sieci Neuronowych (surowe dane):  0.8375
Confusion matrix dla Sieci Neuronowych (surowe dane): 

[[29  0  1  1  0  0  1  0  0  0]
 [ 0 49  2  1  0  0  0  0  1  0]
 [ 1  0 41  1  0  0  0  0  8  1]
 [ 0  0  0 43  0  2  0  0  0  0]
 [ 0  0  0  1 31  0  2  0  1  8]
 [ 0  0  0  1  3 20  0  3  3  1]
 [ 1  0  0  0  0  0 34  0  0  0]
 [ 0  0  4  0  1  0  0 32  1  4]
 [ 0  0  1  3  0  0  0  1 31  1]
 [ 0  0  0  0  0  0  2  2  1 25]]
Classification report dla Sieci Neuronowych (surowe dane): 
              precision    recall  f1-score   support

           0       0.94      0.91      0.92        32
           1       1.00      0.92      0.96        53
           2       0.84      0.79      0.81        52
           3       0.84      0.96      0.90        45
           4       0.89      0.72      0.79        43
           5       0.91      0.65      0.75        31
           6       0.87      0.97      0.92        35
           7       0.84      0.76 

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/sequential.py:450: UserWarning: `model.predict_classes()` is deprecated and will be removed after 2021-01-01. Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
  warnings.warn('`model.predict_classes()` is deprecated and '


Sieci neuronowe dla surowych danych zmieszanych

In [ ]:
model=KerasClassifier(create_model)
batch_size = [5,10, 20]
epochs = [1,5,10,15]

param_grid = dict(batch_size=batch_size, epochs=epochs)
nn_model_shuffle = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1, cv=3)

nn_model_shuffle.fit(permute_train_raw_data_short,encoded_train_labels_short)


Epoch 1/10
30/30 [==============================] - 1s 4ms/step - loss: 223.5914 - accuracy: 0.0667
Epoch 2/10
30/30 [==============================] - 0s 4ms/step - loss: 44.1943 - accuracy: 0.2231
Epoch 3/10
30/30 [==============================] - 0s 5ms/step - loss: 26.2011 - accuracy: 0.3402
Epoch 4/10
30/30 [==============================] - 0s 5ms/step - loss: 18.0330 - accuracy: 0.4727
Epoch 5/10
30/30 [==============================] - 0s 4ms/step - loss: 10.5949 - accuracy: 0.5571
Epoch 6/10
30/30 [==============================] - 0s 4ms/step - loss: 7.6003 - accuracy: 0.6296
Epoch 7/10
30/30 [==============================] - 0s 4ms/step - loss: 6.0165 - accuracy: 0.7303
Epoch 8/10
30/30 [==============================] - 0s 4ms/step - loss: 4.5342 - accuracy: 0.7420
Epoch 9/10
30/30 [==============================] - 0s 5ms/step - loss: 4.0769 - accuracy: 0.8010
Epoch 10/10
30/30 [==============================] - 0s 5ms/step - loss: 4.8121 - accuracy: 0.7946


GridSearchCV(cv=3, error_score=nan,
             estimator=<tensorflow.python.keras.wrappers.scikit_learn.KerasClassifier object at 0x7f2f2712a050>,
             iid='deprecated', n_jobs=-1,
             param_grid={'batch_size': [5, 10, 20], 'epochs': [1, 5, 10, 15]},
             pre_dispatch='2*n_jobs', refit=True, return_train_score=False,
             scoring=None, verbose=0)

In [ ]:
print(nn_model_shuffle.best_params_)
nn_pred_shuffle = nn_model_shuffle.predict(permute_test_raw_data_short)

print("Accuracy dla Sieci neuronowych (surowe dane zmieszane): ", accuracy_score(test_labels_short, nn_pred_shuffle))
print("Confusion matrix dla Sieci neuronowych (surowe dane zmieszane): \n")
print(confusion_matrix(test_labels_short, nn_pred_shuffle))
print("Classification report dla Sieci neuronowych (surowe dane zmieszane): ")
print(classification_report(test_labels_short, nn_pred_shuffle))

{'batch_size': 20, 'epochs': 10}
Accuracy dla Sieci neuronowych (surowe dane zmieszane):  0.1075
Confusion matrix dla Sieci neuronowych (surowe dane zmieszane): 

[[ 6  5  2  7  0  4  2  2  0  4]
 [11  9  3  7  2  7  2  2  3  7]
 [ 5  6  7 13  2  7  2  3  1  6]
 [13  3  7  7  1  5  1  0  1  7]
 [ 4  2  7  4  0  8  0  5  1 12]
 [ 9  2  4  3  3  1  1  1  1  6]
 [ 9  5  9  4  0  3  0  2  0  3]
 [ 6  5  9  9  1  4  2  3  0  3]
 [ 6  7  6  3  1  3  1  3  3  4]
 [ 8  6  2  6  0  1  0  0  0  7]]
Classification report dla Sieci neuronowych (surowe dane zmieszane): 
              precision    recall  f1-score   support

           0       0.08      0.19      0.11        32
           1       0.18      0.17      0.17        53
           2       0.12      0.13      0.13        52
           3       0.11      0.16      0.13        45
           4       0.00      0.00      0.00        43
           5       0.02      0.03      0.03        31
           6       0.00      0.00      0.00        35
   

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/sequential.py:450: UserWarning: `model.predict_classes()` is deprecated and will be removed after 2021-01-01. Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
  warnings.warn('`model.predict_classes()` is deprecated and '


Dla wszystkich klasyfikatorów wyniki po zastosowaniu deskew są lepsze, niż bez deskew. Dla danych zmieszanych dokładność jest najmniejsza.